# Snowflake Metadata Refresh Setup

## Import Python Libraries

In [ ]:
import snowflake.connector
import os
import pandas as pd
import numpy as np
import pandas as pd
import pyarrow as pa
# pd.set_option('max_columns', 40)

## Set Snowflake Variables

In [ ]:
# snowflake connection variables
snowflake_user = 'JMILLER'
snowflake_password = os.environ['BISNOWPASS']
snowflake_account = 'eh69371.east-us-2.azure'
snowflake_role = 'SYSADMIN'
snowflake_warehouse = 'COMPUTE_WH'

# database and schema for metadata objects
snowflake_database = 'UTIL_DB'
snowflake_schema = 'METADATA'

print('Using Notebook Variables:')
print('snowflake_user: ' + snowflake_user)
print('snowflake_password: ' + '***************')
print('snowflake_account: ' + snowflake_account)
print('snowflake_role: ' + snowflake_role)
print('snowflake_warehouse: ' + snowflake_warehouse)
print('snowflake_database: ' + snowflake_database)
print('snowflake_schema: ' + snowflake_schema)

## Establish Snowflake Connection

In [ ]:
ctx = snowflake.connector.connect(
    user = snowflake_user,
    password = snowflake_password,
    account = snowflake_account,
    role = snowflake_role,
    warehouse = snowflake_warehouse
    )
cur = ctx.cursor()

# Return Client
cur.execute("select CURRENT_CLIENT()")
one_row = cur.fetchone()
print('Snowflake Connection Successful')
print(one_row[0])

## Create Database and Schema if they do not exist

In [ ]:
sql = "create database if not exists " + snowflake_database
print(sql)
cur.execute(sql)

sql = "create schema if not exists " + snowflake_database + "." + snowflake_schema
print(sql)
cur.execute(sql)

## Set Snowflake Database and Schema Context

In [ ]:
sql = "use database " + snowflake_database + ";"
print(sql)
cur.execute(sql)

sql = "use schema " + snowflake_schema + ";"
print(sql)
cur.execute(sql)

## Create added column names for Dimensions table if needed

In [ ]:
sql = """
    CREATE TABLE IF NOT EXISTS ADDED_DIM_COLUMN_NAMES (
        ADDED_DIM_COLUMN_NAMES_TAG VARCHAR(255) NOT NULL,
        ROW_IS_CURRENT VARCHAR(255) NOT NULL,
        ROW_EFFECTIVE_DATE VARCHAR(255) NOT NULL,
        ROW_EXPIRATION_DATE VARCHAR(255) NOT NULL,
        ROW_INSERT_DATE VARCHAR(255) NOT NULL,
        ROW_UPDATE_DATE VARCHAR(255) NOT NULL)
"""
cur.execute(sql)
#df = cur.fetch_pandas_all()
#df

## Insert row for ADDED_DIM_COLUMN_NAMES table if needed

In [ ]:
sql = """
	INSERT INTO ADDED_DIM_COLUMN_NAMES
	WITH base (ADDED_DIM_COLUMN_NAMES_TAG,ROW_IS_CURRENT,ROW_EFFECTIVE_DATE,ROW_EXPIRATION_DATE,ROW_INSERT_DATE,ROW_UPDATE_DATE) AS 
	(
	SELECT 'standard_lc','row_is_current','row_effective_date','row_expiration_date','row_insert_date','row_update_date'
	UNION ALL
	SELECT 'standard_uc','ROW_IS_CURRENT','ROW_EFFECTIVE_DATE','ROW_EXPIRATION_DATE','ROW_INSERT_DATE','ROW_UPDATE_DATE'
	)
	SELECT b.ADDED_DIM_COLUMN_NAMES_TAG,
		   b.ROW_IS_CURRENT,
		   b.ROW_EFFECTIVE_DATE,
		   b.ROW_EXPIRATION_DATE,
		   b.ROW_INSERT_DATE,
		   b.ROW_UPDATE_DATE
	  FROM base 
 LEFT JOIN ADDED_DIM_COLUMN_NAMES a
		ON a.ADDED_DIM_COLUMN_NAMES_TAG = b.ADDED_DIM_COLUMN_NAMES_TAG
	 WHERE a.ADDED_DIM_COLUMN_NAMES_TAG IS NULL
"""
cur.execute(sql)
one_row = cur.fetchone()
print('Rows inserted: ', one_row[0])